In [7]:
import torch


conv = torch.nn.Conv2d(3, 3, kernel_size=3, padding=1, bias=False)

# change parameter of conv


input_tensor = torch.tensor([
    [[1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]],

    [[2.0, 3.0, 4.0],
    [5.0, 6.0, 7.0],
    [8.0, 9.0, 1.0]],

    [[3.0, 4.0, 5.0],
    [6.0, 7.0, 8.0],
    [9.0, 1.0, 2.0]],


    
    ])

print(input_tensor.shape)
conv.weight = torch.nn.Parameter(torch.tensor([[
    [[1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]],

    [[2.0, 3.0, 4.0],
    [5.0, 6.0, 7.0],
    [8.0, 9.0, 1.0]],

    [[3.0, 4.0, 5.0],
    [6.0, 7.0, 8.0],
    [9.0, 1.0, 2.0]],

],
[
    [[1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]],

    [[2.0, 3.0, 4.0],
    [5.0, 6.0, 7.0],
    [8.0, 9.0, 1.0]],

    [[3.0, 4.0, 5.0],
    [6.0, 7.0, 8.0],
    [9.0, 1.0, 2.0]],

],
[
    [[1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]],

    [[2.0, 3.0, 4.0],
    [5.0, 6.0, 7.0],
    [8.0, 9.0, 1.0]],

    [[3.0, 4.0, 5.0],
    [6.0, 7.0, 8.0],
    [9.0, 1.0, 2.0]],

]]))


output = conv(input_tensor.unsqueeze(0))  # Add batch dimension

output

torch.Size([3, 3, 3])


tensor([[[[251., 474., 386.],
          [498., 855., 498.],
          [386., 474., 251.]],

         [[251., 474., 386.],
          [498., 855., 498.],
          [386., 474., 251.]],

         [[251., 474., 386.],
          [498., 855., 498.],
          [386., 474., 251.]]]], grad_fn=<ConvolutionBackward0>)

In [ ]:

class MulMask(nn.Module):
    def __init__(self, mask):
        super().__init__()
        self.register_buffer("mask", mask)
    def forward(self, W):
        return W * self.mask


class VisionTransformer(timm.models.vision_transformer.VisionTransformer):
    """ Vision Transformer with support for global average pooling and custom QKV input """
    def __init__(self, global_pool=False, mask_ratio=0.0, **kwargs):
        super().__init__(**kwargs)
        ...
        self.pred_conv = torch.nn.Conv2d(self.d_model, self.d_model, kernel_size=3, stride=1, padding=1)
        self.weight_mask = torch.ones_like(self.pred_conv.weight)
        self.weight_mask[..., 1, 1] = 0
        self.register_buffer("weight_mask", self.weight_mask)  # 학습X, 디바이스 이동/저장O

        parametrize.register_parametrization(self.pred_conv, "weight", MulMask(self.weight_mask))


    def calc_energy(self, x): # x shape: batch_num, patch_num, emb_dim
        H, W = self.img_size
        B, N, D = x.shape

        H_p, W_p = self.patch_embed.grid_size
        if N == 1 + H_p * W_p:
            x = x[:, 1:, :]
            N = H_p * W_p
        x_2d = x.reshape(B, H_p, W_p, D).permute(0, 3, 1, 2).contiguous()

        # 패치별 에너지 계산 (MAE 스타일)

        out_2d = self.pred_conv(x_2d)  # (B, D, H_p, W_p)

        out = out_2d.permute(0, 2, 3, 1).reshape(B, N, D)
        energy = F.cosine_similarity(x.float(), out.float(), dim=-1)  # (B, N)

        return energy

torch.Size([2, 10, 10])

In [16]:
H, W = 40, 40
# i+j 홀수인 마스크 생성
i_idx = torch.arange(H).unsqueeze(1)
j_idx = torch.arange(W).unsqueeze(0)
mask1 = (i_idx + j_idx) % 2 == 1  # dtype: bool
mask2 = ~mask1

checker_masking = torch.stack([mask1, mask2])
checker_masking.shape

torch.Size([2, 40, 40])

In [ ]:
import random

mask_ratio = 0.01
true_indices = checker_masking[random.randint(0,1)].nonzero(as_tuple=False)


16

In [112]:
def odd_sum_flat_indices(W: int, H: int, device=None):
    """
    (W, H) 격자에서 (i + j)가 홀수인 위치를
    flatten(view(-1)) 순서(행우선, flat_idx = i*H + j) 기준으로 반환.

    Returns:
        mask_flat: (W*H,) bool  — i+j가 홀수면 True
        idx_flat:  (K,)   long  — 위 조건을 만족하는 평탄화 인덱스
    """
    device = device or "cpu"

    # (W,H)에서 i=행(0..W-1), j=열(0..H-1)
    i = torch.arange(W, device=device).unsqueeze(1)   # (W,1)
    j = torch.arange(H, device=device).unsqueeze(0)   # (1,H)

    # 2D 마스크: i+j가 홀수
    odd2d = ((i + j) % 2 == 0)                        # (W,H) bool

    # flatten(view) 순서로 1D 마스크/인덱스
    mask_flat = odd2d.view(-1)                        # (W*H,)
    idx_flat = mask_flat.nonzero(as_tuple=False).squeeze(1)   # (K,)
    return idx_flat


W, H = 6, 6

my_data = torch.ones((W, H), dtype=torch.int16)
idx = odd_sum_flat_indices(W, H)
my_data_flatten = my_data.flatten()

my_data_flatten[idx] = 0 
my_data = my_data_flatten.view(W, H)  # (W, H) 형태로 복원
my_data


tensor([[0, 1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1, 0]], dtype=torch.int16)

In [77]:

my_data = torch.randint(0, 10, (W, H), dtype=torch.int16).flatten().unsqueeze(0)
mask_idx = odd_sum_flat_indices(W, H)
print(my_data)
print(my_data[:, mask_idx])

tensor([[1, 9, 4, 6, 5, 6, 0, 2, 2, 4, 6, 0, 6, 5, 6, 2, 3, 1, 3, 9, 5, 8, 3, 6,
         0, 8, 5, 9, 9, 9, 3, 4, 3, 6, 0, 1]], dtype=torch.int16)
tensor([[9, 6, 6, 0, 2, 6, 5, 2, 1, 3, 5, 3, 8, 9, 9, 3, 3, 0]],
       dtype=torch.int16)


In [142]:
def masked_topk(X: torch.Tensor, cand_mask: torch.Tensor, k: int, largest: bool):
    """
    X: (B, N)
    cand_mask: (B, N) 또는 (N,)  True = 후보로 포함
    k: 뽑을 개수
    largest: True면 상위 K, False면 하위 K
    return: vals, idx  (둘 다 (B, k))
    """
    if cand_mask.dim() == 1:
        cand_mask = cand_mask.unsqueeze(0).expand(X.size(0), -1)

    sentinel = float('-inf') if largest else float('inf')
    X_masked = X.masked_fill(~cand_mask, sentinel)
    vals, idx = torch.topk(X_masked, k=k, dim=1, largest=largest, sorted=True)
    return vals, idx



mask_ratio = 0.6
keep_ratio = 1 - mask_ratio
my_data = torch.tensor([
    [1, 5, 2, 
     -1, 5, 6, 
     7, 8, 9],
                        [1, 5, 2, -1, 5, 6, -7, 8, 9]])


idx = odd_sum_flat_indices(3, 3)


my_data_choice = my_data.clone()
my_data_choice[:, idx] = 9999
# my_data_choice[:, idx] = float('inf')


_, keep_idx = torch.topk(my_data_choice, k=int((len(my_data[0]) - len(idx)) * (1 - mask_ratio*2)) + len(idx), dim=1)
keep_idx

tensor([[4, 8, 6, 0, 2],
        [4, 8, 6, 0, 2]])

In [133]:
int((len(my_data) - len(idx)) * (1 - mask_ratio*2))# + len(idx)

-3

In [115]:
my_data[:, idx] *= 100000000
my_data

tensor([[ 100000000,          5,  200000000,         -1,  500000000,          6,
          700000000,          8,  900000000],
        [ 100000000,          5,  200000000,         -1,  500000000,          6,
         -700000000,          8,  900000000]])

In [103]:
my_data_kept = my_data.gather(1, keep_idx)  # (B, N-K)
my_data_kept

tensor([[10,  9,  8,  7,  6,  5],
        [ 9,  8,  6,  5,  5,  2]])

In [149]:
x = torch.randn(5, 6, 7)  # (B, N, D)


idx = torch.tensor([
    [0, 1],
    [2, 3]
]) # (B, N')

keep_mask = torch.zeros(len(x[1]), dtype=torch.bool)
keep_mask[idx] = True



x_ = x[:,keep_mask, :]  # (B, K, D)
x_.shape

torch.Size([5, 4, 7])

In [150]:
print(x)

tensor([[[ 0.5404,  1.5642,  0.7286,  0.7942,  1.2502, -0.3675, -0.8182],
         [ 0.7306,  0.2557, -0.5653,  0.0101,  0.8182,  1.1749, -0.7822],
         [-0.5330, -0.4039,  0.7959, -3.6322,  0.7030, -1.2678,  1.2280],
         [ 0.3460, -0.1492, -0.5937, -0.4439,  0.0613, -0.3712,  0.1414],
         [-0.8871,  0.6600,  0.5419, -0.3703,  0.1059, -1.0887,  1.2211],
         [ 0.5977,  1.1057, -0.2580,  0.4664, -0.8103, -0.2414, -0.9857]],

        [[-0.2050, -0.3072, -1.4555,  1.0356, -1.3507, -1.2357, -1.4040],
         [-0.8913, -1.0701,  1.0238, -0.7172,  0.9297,  0.5454, -1.8978],
         [-0.3596,  0.5746, -1.4057, -0.8708, -0.2433, -0.8615,  1.1718],
         [-0.0807,  0.1792, -1.0630, -0.7052, -0.1952, -0.1136, -0.0244],
         [ 0.2372, -0.5564, -1.8119, -1.3283,  0.8437,  0.3541, -0.0890],
         [-0.2144,  0.3791, -0.4306,  0.2335, -0.5746,  0.4505, -0.0391]],

        [[-1.5534,  1.2160, -1.6798,  0.2679,  2.4092,  0.4152,  0.1347],
         [-0.1756,  1.6643, -1.455

In [151]:
print(x_)

tensor([[[ 0.5404,  1.5642,  0.7286,  0.7942,  1.2502, -0.3675, -0.8182],
         [ 0.7306,  0.2557, -0.5653,  0.0101,  0.8182,  1.1749, -0.7822],
         [-0.5330, -0.4039,  0.7959, -3.6322,  0.7030, -1.2678,  1.2280],
         [ 0.3460, -0.1492, -0.5937, -0.4439,  0.0613, -0.3712,  0.1414]],

        [[-0.2050, -0.3072, -1.4555,  1.0356, -1.3507, -1.2357, -1.4040],
         [-0.8913, -1.0701,  1.0238, -0.7172,  0.9297,  0.5454, -1.8978],
         [-0.3596,  0.5746, -1.4057, -0.8708, -0.2433, -0.8615,  1.1718],
         [-0.0807,  0.1792, -1.0630, -0.7052, -0.1952, -0.1136, -0.0244]],

        [[-1.5534,  1.2160, -1.6798,  0.2679,  2.4092,  0.4152,  0.1347],
         [-0.1756,  1.6643, -1.4552,  2.2137,  1.1880,  0.3037,  0.1410],
         [-0.5269,  1.8338, -2.0599,  1.7139, -1.2064, -0.1879, -1.5944],
         [ 1.1134,  0.6083,  0.2002, -1.0417,  1.2494, -1.1443, -2.6373]],

        [[-1.4631,  0.1591, -0.7854,  0.0742, -0.7529,  0.7044, -0.1008],
         [-2.1103,  0.4103,  0.7